# Imports & dependencies

In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch_geometric.transforms as T
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear
import torch.optim as optim
# # resources:
# 1.https://pytorch-geometric.readthedocs.io/en/2.0.0/notes/heterogeneous.html?highlight=HeteroGNN#using-the-heterogenous-convolution-wrapper 
# 2.https://levelup.gitconnected.com/forecasting-walmart-quarterly-revenue-pytorch-lstm-example-b4e4b20862a7

In [2]:
import networkx as nx
def A_to_edge_index(A):
    G=nx.from_numpy_matrix(A)
    edge_index=list(G.edges)
    z=torch.tensor(np.transpose(edge_index))
    return z

# Reading the data

In [3]:
import pandas as pd
import numpy as np
a=pd.read_pickle('data/adj_data/adj_mx_taxi.pkl')
adj=np.array(a)

# 1. load the data from the pytorch files:
adj=torch.load('adj.pt')
adj2=adj[2]
# convert it to int
adj2 = (np.ceil(adj2)).astype(int)
adj1=adj[1]
# convert it to int
# adj1 = (np.ceil(adj1)).astype(int)

x_train=torch.load('x_train.pt')
y_train=torch.load('y_train.pt')
x_test=torch.load('x_test.pt')
y_test=torch.load('y_test.pt')

/home/lambda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


In [4]:
# 2. load the saved 4 tesnors: input/output, training and testing
xtrain=torch.load('xtrain.pt')
ytrain=torch.load('ytrain.pt')
xtest=torch.load('xtest.pt')
ytest=torch.load('ytest.pt')
# verbose
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

torch.Size([23940, 24])
torch.Size([23940, 24])
torch.Size([5320, 24])
torch.Size([5320, 24])


In [5]:
# 3. convert the adj  matrix to adjacency list
edge_list0=A_to_edge_index(adj2)
edge_list2=A_to_edge_index(adj2)

In [6]:
# Creating a Heterogeneous Graph
from torch_geometric.data import HeteroData
data = HeteroData()
data['taxi'].x= xtest[0:266, :] 
data['taxi', 'near', 'taxi'].edge_index = edge_list0
data['taxi', 'connected', 'taxi'].edge_index = edge_list2
data['taxi', 'OD_similar', 'taxi'].edge_index = edge_list2
temp=np.array([True])
temp2=np.tile(temp, 266)
data['taxi'].test_mask=torch.from_numpy(temp2)

In [7]:
print(data)

HeteroData(
  taxi={
    x=[266, 24],
    test_mask=[266]
  },
  (taxi, near, taxi)={ edge_index=[2, 10064] },
  (taxi, connected, taxi)={ edge_index=[2, 10064] },
  (taxi, OD_similar, taxi)={ edge_index=[2, 10064] }
)


In [8]:
node_types, edge_types = data.metadata()
print("Node types:", node_types)
print("Edge types:", edge_types)

Node types: ['taxi']
Edge types: [('taxi', 'near', 'taxi'), ('taxi', 'connected', 'taxi'), ('taxi', 'OD_similar', 'taxi')]


# The GNN Model

In [9]:
class HeteroGNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_layers):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                ('taxi', 'near', 'taxi'): GCNConv(-1, hidden_channels),
                ('taxi', 'connected', 'taxi'): SAGEConv((-1, -1), hidden_channels),
                ('taxi', 'OD_similar', 'taxi'): GATConv((-1, -1), hidden_channels),
            }, aggr='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: x.relu() for key, x in x_dict.items()}
        return self.lin(x_dict['taxi'])



In [10]:
Model_GNN = HeteroGNN(hidden_channels=64, out_channels=24,
                  num_layers=2)

print(Model_GNN)
with torch.no_grad():  # Initialize lazy modules.
     out = Model_GNN(data.x_dict, data.edge_index_dict)
        
print(out.shape)

HeteroGNN(
  (convs): ModuleList(
    (0): HeteroConv(num_relations=3)
    (1): HeteroConv(num_relations=3)
  )
  (lin): Linear(64, 24, bias=True)
)
torch.Size([266, 24])


### The LSTM model

In [11]:
class LSTM(nn.Module):
    """
    input_size - will be 1 in this example since we have only 1 predictor (a sequence of previous values)
    hidden_size - Can be chosen to dictate how much hidden "long term memory" the network will have
    output_size - This will be equal to the prediciton_periods input to get_x_y_pairs
    """
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size)
        
        self.linear = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hidden=None):
        if hidden==None:
            self.hidden = (torch.zeros(1,1,self.hidden_size),
                           torch.zeros(1,1,self.hidden_size))
        else:
            self.hidden = hidden
            
        """
        inputs need to be in the right shape as defined in documentation
        - https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
        
        lstm_out - will contain the hidden states from all times in the sequence
        self.hidden - will contain the current hidden state and cell state
        """
        lstm_out, self.hidden = self.lstm(x.view(len(x),1,-1), 
                                          self.hidden)
        
        predictions = self.linear(lstm_out.view(len(x), -1))
        
        return predictions[-1], self.hidden

In [12]:
model_LSTM = LSTM(input_size=1, hidden_size=50, output_size=8)
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model_LSTM.parameters(), lr=0.001)